___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___

# WELCOME!

Welcome to "***Car Price Prediction Project***". This is the first medium project of ***Machine Learning*** course. In this project you will have the opportunity to apply many algorithms commonly used for regression problems.

Also, you will apply various processes such as pre-processing, ***train-test spilit*** and ***cross validation*** that you will use in algorithm modeling and prediction processes in Python with ***scikit-learn***. 

Before diving into the project, please take a look at the determines and tasks.

- **NOTE:** This project assumes that you already know the basics of coding in Python. You should also be familiar with the theory behind regression algorithms and scikit-learn module as well as Machine Learning before you begin.


# #Determines

**Auto Scout** data which using for this project, scraped from the on-line car trading company(https://www.autoscout24.com)in 2019, contains many features of 9 different car models. In this project, you will use the data set which is already preprocessed and prepared for algorithms .

The aim of this project to understand of machine learning algorithms. Therefore, you will not need any EDA process as you will be working on the edited data.

---

In this Senario, you will estimate the prices of cars using regression algorithms.

While starting you should import the necessary modules and load the data given as pkl file. Also you'll need to do a few pre-processing before moving to modelling. After that you will implement ***Linear Regression, Ridge Regression, Lasso Regression,and Elastic-Net algorithms respectively*** (After completion of Unsupervised Learning section, you can also add bagging and boosting algorithms such as ***Random Forest and XG Boost*** this notebook to develop the project). You can measure the success of your models with regression evaluation metrics as well as with cross validation method.

For the better results, you should try to increase the success of your models by performing hyperparameter tuning. Determine feature importances for the model. You can set your model with the most important features for resource saving. You should try to apply this especially in Random Forest and XG Boost algorithms. Unlike the others, you will perform hyperparameter tuning for Random Forest and XG Boost using the ***GridSearchCV*** method. 

Finally You can compare the performances of algorithms, work more on the algorithm have the most successful prediction rate.






# #Tasks

#### 1. Import Modules, Load Data and Data Review
#### 2. Data Pre-Processing
#### 3. Implement Linear Regression 
#### 4. Implement Ridge Regression
#### 5. Implement Lasso Regression 
#### 6. Implement Elastic-Net
#### 7. Visually Compare Models Performance In a Graph

## 1. Import Modules, Load Data and Data Review

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import SCORERS
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.model_selection import GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (10,6)
pd.set_option('display.max_columns', 175)

In [48]:
pd.set_option('display.max_rows', 175)

In [49]:
# function to compute adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute MAPE
def mape_score(targets, predictions):
    return np.mean(np.abs(targets - predictions) / targets) * 100


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mape_score(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

In [85]:
def train_val(y_train, y_train_pred, y_test, y_pred, i):
    
    scores = {i+"_train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    i+"_test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [50]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [51]:
df = pd.read_csv('scout_final_g2.csv')

In [52]:
df

,Unnamed: 0,price,km,car_age,prev_owner,horse_power,gears,nr_of_doors,nr_of_seats,displacement,cylinders,weight_kg,consump_comb,consump_city,consump_country,co2_emission,cc_Air conditioning,cc_Air suspension,cc_Armrest,cc_Automatic climate control,cc_Auxiliary heating,cc_Cruise control,cc_Electric Starter,cc_Electric tailgate,cc_Electrical side mirrors,cc_Electrically adjustable seats,cc_Electrically heated windshield,cc_Heads-up display,cc_Heated steering wheel,cc_Hill Holder,cc_Keyless central door lock,cc_Leather seats,cc_Leather steering wheel,cc_Light sensor,cc_Lumbar support,cc_Massage seats,cc_Multi-function steering wheel,cc_Navigation system,cc_Panorama roof,cc_Park Distance Control,cc_Parking assist system camera,cc_Parking assist system self-steering,cc_Parking assist system sensors front,cc_Parking assist system sensors rear,cc_Power windows,cc_Rain sensor,cc_Seat heating,cc_Seat ventilation,cc_Split rear seats,cc_Start-stop system,cc_Sunroof,cc_Tinted windows,cc_Wind deflector,cc_Windshield,em_Bluetooth,em_CD player,em_Digital radio,em_Hands-free equipment,em_MP3,em_On-board computer,em_Radio,em_Sound system,em_Television,em_USB,ex_Alloy wheels,ex_Cab or rented Car,ex_Catalytic Converter,ex_Handicapped enabled,ex_Right hand drive,ex_Roof rack,ex_Shift paddles,ex_Ski bag,ex_Sliding door,ex_Sport package,ex_Sport seats,ex_Sport suspension,ex_Touch screen,ex_Trailer hitch,ex_Tuned car,ex_Voice Control,ex_Winter tyres,ss_abs,ss_adaptive cruise control,ss_adaptive headlights,ss_alarm system,ss_blind spot monitor,ss_central door lock,ss_central door lock with remote control,ss_daytime running lights,ss_driver drowsiness detection,ss_driver-side airbag,ss_electronic stability control,ss_emergency brake assistant,ss_emergency system,ss_fog lights,ss_head airbag,ss_immobilizer,ss_isofix,ss_lane departure warning system,ss_led daytime running lights,ss_led headlights,ss_night view assist,ss_passenger-side airbag,ss_power steering,ss_rear airbag,ss_side airbag,ss_tire pressure monitoring system,ss_traction control,ss_traffic sign recognition,ss_xenon headlights,make_model_Audi A1,make_model_Audi A2,make_model_Audi A3,make_model_Opel Astra,make_model_Opel Corsa,make_model_Opel Insignia,make_model_Renault Clio,make_model_Renault Duster,make_model_Renault Espace,body_type_Compact,body_type_Convertible,body_type_Coupe,body_type_Off-Road,body_type_Other,body_type_Sedans,body_type_Station wagon,body_type_Transporter,body_type_Van,color_Beige,color_Black,color_Blue,color_Bronze,color_Brown,color_Green,color_Grey,color_Orange,color_Red,color_Silver,color_Violet,color_White,color_Yellow,paint_type_Metallic,paint_type_Perl,paint_type_Uni,type_Demonstration,type_Employee's car,type_New,type_Pre-registered,type_Used,upholstery_Beige,upholstery_Black,upholstery_Blue,upholstery_Brown,upholstery_Cloth,upholstery_Full leather,upholstery_Grey,upholstery_Other,upholstery_Part leather,upholstery_Velour,upholstery_White,upholstery_alcantara,gearing_type_Automatic,gearing_type_Manual,gearing_type_Semi-automatic,drive_chain_4WD,drive_chain_front,drive_chain_rear,fuel_Benzine,fuel_Diesel,fuel_Electric,fuel_LPG/CNG
0,0,15770,56013.000000,3,2,66.0,7.0,5.0,5,1400.0,3.0,1220.0,3.8,4.3,3.5,99.0,1,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0
1,1,14500,80000.000000,2,1,141.0,7.0,3.0,4,1800.0,4.0,1255.0,5.6,7.1,4.7,129.0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,1,1,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0
2,2,14640,83450.000000,3,1,85.0,7.0,4.0,4,1600.0,4.0,1135.0,3.8,4.4,3.4,9

In [53]:
del(df['Unnamed: 0'])

In [54]:
df.columns

Index(['price', 'km', 'car_age', 'prev_owner', 'horse_power', 'gears',
       'nr_of_doors', 'nr_of_seats', 'displacement', 'cylinders',
       ...
       'gearing_type_Automatic', 'gearing_type_Manual',
       'gearing_type_Semi-automatic', 'drive_chain_4WD', 'drive_chain_front',
       'drive_chain_rear', 'fuel_Benzine', 'fuel_Diesel', 'fuel_Electric',
       'fuel_LPG/CNG'],
      dtype='object', length=170)

In [55]:
df.shape

(15915, 170)

In [56]:
df.isnull().sum()

price                                       0
km                                          0
car_age                                     0
prev_owner                                  0
horse_power                                 0
gears                                       0
nr_of_doors                                 0
nr_of_seats                                 0
displacement                                0
cylinders                                   0
weight_kg                                   0
consump_comb                                0
consump_city                                0
consump_country                             0
co2_emission                                0
cc_Air conditioning                         0
cc_Air suspension                           0
cc_Armrest                                  0
cc_Automatic climate control                0
cc_Auxiliary heating                        0
cc_Cruise control                           0
cc_Electric Starter               

## 2. Data Pre-Processing

As you know, the data set must be edited before proceeding to the implementation of the model. As the last step before model fitting, you need to spilit the data set as train and test. Then, you should train the model with train data and evaluate the performance of the model on the test data. You can use the train and test data you have created for all algorithms.

You must also drop your target variable, the column you are trying to predict.

You can use many [performance metrics for regression](https://medium.com/analytics-vidhya/evaluation-metrics-for-regression-problems-343c4923d922) to measure the performance of the regression model you train. You can define a function to view different metric results together.

You can also use the [cross validation](https://towardsdatascience.com/cross-validation-explained-evaluating-estimator-performance-e51e5430ff85) method to measure the estimator performance. Cross validation uses different data samples from your test set and calculates the accuracy score for each data sample. You can calculate the final performance of your estimator by averaging these scores.

In [57]:
X = df.drop("price", axis =1)
y = df["price"]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [59]:
#scaler = StandardScaler()

In [60]:
#scaler.fit(X_train)  

In [61]:
#X_train_scaled = scaler.transform(X_train)
#X_train_scaled

In [62]:
#X_test_scaled = scaler.transform(X_test)
#X_test_scaled

## 3. Implement Linear Regression

 - Import the modul
 - Fit the model 
 - Predict the test set
 - Determine feature coefficiant
 - Evaluate model performance (use performance metrics for regression and cross_val_score)
 - Compare different evaluation metrics
 
*Note: You can use the [dir()](https://www.geeksforgeeks.org/python-dir-function/) function to see the methods you need.*

In [63]:
linearregression = LinearRegression()

In [64]:
linearregression.fit(X_train, y_train)

LinearRegression()

In [86]:
y_pred = linearregression.predict(X_test)
y_train_pred = linearregression.predict(X_train)

In [66]:
print("Training Performance\n")
linearregression_train_perf = model_performance_regression(
    linearregression, X_train, y_train
)
linearregression_train_perf

Training Performance



,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,2375.216029,1655.933384,0.896523,0.894929,10.041017


In [67]:
print("Test Performance\n")
linearregression_test_perf = model_performance_regression(
    linearregression, X_test, y_test
)
linearregression_test_perf

Test Performance



,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,2455.029632,1713.568067,0.889215,0.885149,10.386512


In [87]:
ls =train_val(y_train, y_train_pred, y_test, y_pred, "linear")
ls

,linear_train,linear_test
R2,8.965229e-01,8.892145e-01
mae,1.655933e+03,1.713568e+03
mse,5.641651e+06,6.027170e+06
rmse,2.375216e+03,2.455030e+03


### Cross_validate


In [68]:
model = LinearRegression()
scores = cross_validate(model, X_train, y_train, scoring = ['r2', 'neg_mean_absolute_error','neg_mean_squared_error', \
                                                            'neg_root_mean_squared_error'], cv = 5)

In [69]:
scores

{'fit_time': array([0.06205535, 0.06806254, 0.06505895, 0.06505895, 0.06505919]),
 'score_time': array([0.00600553, 0.00500417, 0.00500393, 0.00600553, 0.00400376]),
 'test_r2': array([0.88816843, 0.88744488, 0.89372612, 0.89344945, 0.88969928]),
 'test_neg_mean_absolute_error': array([-1717.66009467, -1710.0894183 , -1622.434475  , -1735.75655537,
        -1665.95999933]),
 'test_neg_mean_squared_error': array([-6081786.03556495, -5953124.50984222, -5325568.35235827,
        -6632492.86480466, -5833133.47808928]),
 'test_neg_root_mean_squared_error': array([-2466.12774113, -2439.90256155, -2307.7192967 , -2575.36266666,
        -2415.18808338])}

In [70]:
pd.DataFrame(scores, index = range(1,6))

,fit_time,score_time,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
1,0.062055,0.006006,0.888168,-1717.660095,-6.081786e+06,-2466.127741
2,0.068063,0.005004,0.887445,-1710.089418,-5.953125e+06,-2439.902562
3,0.065059,0.005004,0.893726,-1622.434475,-5.325568e+06,-2307.719297
4,0.065059,0.006006,0.893449,-1735.756555,-6.632493e+06,-2575.362667
5,0.065059,0.004004,0.889699,-1665.959999,-5.833133e+06,-2415.188083


In [71]:
scores = pd.DataFrame(scores, index=range(1,6))
scores.iloc[:, 2:].mean()

test_r2                             8.904976e-01
test_neg_mean_absolute_error       -1.690380e+03
test_neg_mean_squared_error        -5.965221e+06
test_neg_root_mean_squared_error   -2.440860e+03
dtype: float64

In [72]:
print("Training Performance\n")
linearregression_train_perf = model_performance_regression(
    linearregression, X_train, y_train
)
linearregression_train_perf

Training Performance



,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,2375.216029,1655.933384,0.896523,0.894929,10.041017


In [73]:
print("Testing Performance\n")
linearregression_train_perf = model_performance_regression(
    linearregression, X_test, y_test
)
linearregression_train_perf

Testing Performance



,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,2455.029632,1713.568067,0.889215,0.885149,10.386512


In [88]:
lm_df = pd.DataFrame(linearregression.coef_, columns = ["lm_coef"])
#lm_df

## 4. Implement Ridge Regression

- Import the modul 
- Do not forget to scale the data or use Normalize parameter as True 
- Fit the model 
- Predict the test set 
- Evaluate model performance (use performance metrics for regression) 
- Tune alpha hiperparameter by using [cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) and determine the optimal alpha value.
- Fit the model and predict again with the new alpha value. 

In [75]:
alpha_space = np.linspace(0.01, 1, 100)

In [76]:
ridge_cv_model = RidgeCV(alphas=alpha_space, cv = 5, scoring= "neg_root_mean_squared_error",normalize=True)

In [77]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ]),
        cv=5, normalize=True, scoring='neg_root_mean_squared_error')

In [80]:
ridge_cv_model.alpha_

0.01

In [81]:
ridge_cv_model.best_score_

-2440.6703806426494

In [93]:
# print("Training Performance for Ridge\n")
# linearregression_train_perf = model_performance_regression(
#     ridge_cv_model, X_train, y_train
# )
# linearregression_train_perf

In [94]:
# print("Testing Performance for Ridge\n")
# linearregression_train_perf = model_performance_regression(
#     ridge_cv_model, X_train, y_train
# )
# linearregression_train_perf

In [89]:
y_pred = ridge_cv_model.predict(X_test)
y_train_pred = ridge_cv_model.predict(X_train)

In [95]:
rs =train_val(y_train, y_train_pred, y_test, y_pred, "ridge_cv")
rs

,ridge_cv_train,ridge_cv_test
R2,8.964011e-01,8.895025e-01
mae,1.653325e+03,1.708864e+03
mse,5.648291e+06,6.011505e+06
rmse,2.376613e+03,2.451837e+03


In [96]:
pd.concat([ls, rs], axis = 1)

,linear_train,linear_test,ridge_cv_train,ridge_cv_test
R2,8.965229e-01,8.892145e-01,8.964011e-01,8.895025e-01
mae,1.655933e+03,1.713568e+03,1.653325e+03,1.708864e+03
mse,5.641651e+06,6.027170e+06,5.648291e+06,6.011505e+06
rmse,2.375216e+03,2.455030e+03,2.376613e+03,2.451837e+03


In [111]:
model_performance_regression(ridge_cv_model,X_test,y_test)

,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,2451.837136,1708.863788,0.889502,0.885447,10.341779


In [151]:
(ridge_cv_model.coef_ == 0).sum()

1

## 5. Implement Lasso Regression

- Import the modul 
- Do not forget to scale the data or use Normalize parameter as True(If needed)
- Fit the model 
- Predict the test set 
- Evaluate model performance (use performance metrics for regression) 
- Tune alpha hyperparameter by using [cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) and determine the optimal alpha value.
- Fit the model and predict again with the new alpha value.
- Compare different evaluation metrics

*Note: To understand the importance of the alpha hyperparameter, you can observe the effects of different alpha values on feature coefficants.*

In [98]:
lasso_cv_model = LassoCV(alphas = alpha_space, cv = 5, max_iter=100000, random_state=42,normalize=True) 

In [99]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(alphas=array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ]),
        cv=5, max_iter=100000, normalize=True, random_state=42)

In [100]:
lasso_cv_model.alpha_

0.09999999999999999

In [105]:
y_pred = lasso_cv_model.predict(X_test)   #Lasso(alpha =0.09)
y_train_pred = lasso_cv_model.predict(X_train)

In [106]:
lcs = train_val(y_train, y_train_pred, y_test, y_pred, "lasso_cv")
lcs

,lasso_cv_train,lasso_cv_test
R2,8.959590e-01,8.897615e-01
mae,1.653292e+03,1.706678e+03
mse,5.672394e+06,5.997417e+06
rmse,2.381679e+03,2.448962e+03


In [107]:
pd.concat([ls,rs,lcs], axis = 1)

,linear_train,linear_test,ridge_cv_train,ridge_cv_test,lasso_cv_train,lasso_cv_test
R2,8.965229e-01,8.892145e-01,8.964011e-01,8.895025e-01,8.959590e-01,8.897615e-01
mae,1.655933e+03,1.713568e+03,1.653325e+03,1.708864e+03,1.653292e+03,1.706678e+03
mse,5.641651e+06,6.027170e+06,5.648291e+06,6.011505e+06,5.672394e+06,5.997417e+06
rmse,2.375216e+03,2.455030e+03,2.376613e+03,2.451837e+03,2.381679e+03,2.448962e+03


In [110]:
model_performance_regression(lasso_cv_model,X_test,y_test)

,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,2448.962348,1706.677712,0.889761,0.885716,10.348613


In [150]:
(lasso_cv_model.coef_ == 0).sum()

36

## 6. Implement Elastic-Net

- Import the modul 
- Do not forget to scale the data or use Normalize parameter as True(If needed)
- Fit the model 
- Predict the test set 
- Evaluate model performance (use performance metrics for regression) 
- Tune alpha hyperparameter by using [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and determine the optimal alpha value.
- Fit the model and predict again with the new alpha value.
- Compare different evaluation metrics

In [112]:
elastic_cv_model = ElasticNetCV(alphas = alpha_space, l1_ratio=[0.1, 0.5, 0.7,0.9, 0.95, 1], cv = 5, 
                                max_iter = 100000,random_state=42,normalize=True)

In [113]:
elastic_cv_model.fit(X_train, y_train)

ElasticNetCV(alphas=array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ]),
             cv=5, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 1], max_iter=100000,
             normalize=True, random_state=42)

In [114]:
elastic_cv_model.alpha_

0.09999999999999999

In [115]:
elastic_cv_model.l1_ratio_

1.0

In [116]:
y_pred = elastic_cv_model.predict(X_test)
y_train_pred = elastic_cv_model.predict(X_train)

In [117]:
ecs = train_val(y_train, y_train_pred, y_test, y_pred, "elastic_cv")
ecs

,elastic_cv_train,elastic_cv_test
R2,8.959590e-01,8.897615e-01
mae,1.653292e+03,1.706678e+03
mse,5.672394e+06,5.997417e+06
rmse,2.381679e+03,2.448962e+03


In [119]:
pd.concat([ls,rs,lcs,ecs], axis = 1)

,linear_train,linear_test,ridge_cv_train,ridge_cv_test,lasso_cv_train,lasso_cv_test,elastic_cv_train,elastic_cv_test
R2,8.965229e-01,8.892145e-01,8.964011e-01,8.895025e-01,8.959590e-01,8.897615e-01,8.959590e-01,8.897615e-01
mae,1.655933e+03,1.713568e+03,1.653325e+03,1.708864e+03,1.653292e+03,1.706678e+03,1.653292e+03,1.706678e+03
mse,5.641651e+06,6.027170e+06,5.648291e+06,6.011505e+06,5.672394e+06,5.997417e+06,5.672394e+06,5.997417e+06
rmse,2.375216e+03,2.455030e+03,2.376613e+03,2.451837e+03,2.381679e+03,2.448962e+03,2.381679e+03,2.448962e+03


In [121]:
model_performance_regression(elastic_cv_model,X_test,y_test)

,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,2448.962348,1706.677712,0.889761,0.885716,10.348613


### Grid Search

In [122]:
elastic_model = ElasticNet(max_iter=10000, random_state=42,normalize=True)

In [123]:
param_grid = {"alpha":[0.01, 0.012, 0.2, 0.5, 0.6, 0.7, 1],
            "l1_ratio":[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]}

In [125]:
grid_model = GridSearchCV(estimator = elastic_model, param_grid = param_grid, scoring = 'neg_root_mean_squared_error',
                         cv =5, verbose =2)

In [126]:
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 49 candidates, totalling 245 fits
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.7; total time=   0.0s
[CV] END ...........................alpha=0.01,

GridSearchCV(cv=5,
             estimator=ElasticNet(max_iter=10000, normalize=True,
                                  random_state=42),
             param_grid={'alpha': [0.01, 0.012, 0.2, 0.5, 0.6, 0.7, 1],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_root_mean_squared_error', verbose=2)

###  Using Best Hyper Parameters From GridSearch

In [129]:
y_pred = grid_model.predict(X_test)
y_train_pred = grid_model.predict(X_train)

In [135]:
gr = train_val(y_train, y_train_pred, y_test, y_pred, "GridSearch")
gr

,GridSearch_train,GridSearch_test
R2,8.965132e-01,8.894244e-01
mae,1.655253e+03,1.711995e+03
mse,5.642181e+06,6.015752e+06
rmse,2.375328e+03,2.452703e+03


In [134]:
model_performance_regression(grid_model,X_test,y_test)

,RMSE,MAE,R-squared,Adj. R-squared,MAPE
0,2452.703068,1711.995148,0.889424,0.885366,10.378321


In [136]:
pd.concat([ecs,gr], axis = 1)

,elastic_cv_train,elastic_cv_test,GridSearch_train,GridSearch_test
R2,8.959590e-01,8.897615e-01,8.965132e-01,8.894244e-01
mae,1.653292e+03,1.706678e+03,1.655253e+03,1.711995e+03
mse,5.672394e+06,5.997417e+06,5.642181e+06,6.015752e+06
rmse,2.381679e+03,2.448962e+03,2.375328e+03,2.452703e+03


In [155]:
grid_model.best_params_

{'alpha': 0.012, 'l1_ratio': 1}

## 7. Visually Compare Models Performance In a Graph

___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___